In [1]:
import sys
import os
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel, rbf_kernel
import time
from sklearn.model_selection import train_test_split


# Add parent directory to python path
#PACKAGE_PARENT = '..'
#SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser(__file__))))
#sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
sys.path.append(r'E:\Developer\Python\Myworkshop\Python_Study\机器学习\Matrix Factorization\Jupyter_notebook调试代码')
from my_data_set import DataSet


class GpMf():
    def __init__(self, latent_dim, nb_data):
        self.latent_dim = latent_dim
        self.nb_data = nb_data
        #self.X = np.random.normal(0, 1e-3, (nb_data, latent_dim))   #a spherical Gaussian prior over the X
        self.X = np.random.normal(0, 1e-2, (nb_data, latent_dim))   #a spherical Gaussian prior over the X
        print("self.X:",self.X,"self.X.shape:",self.X.shape)
        self.lin_variance = 1.0
        self.bias_variance = 0.11
        self.white_variance = 5.0
        self.y = None
        self.rated_items = None

    def log_likelihood(self):
        """return the log likelihood of the model"""
        Cj_invy, logDetC = self.invert_covariance()
        yj = np.asmatrix(self.y).T
        Nj = len(self.rated_items)
        #P9 公式20
        likelihood = - 0.5 * (Nj * np.log(2 * math.pi) + logDetC + yj.T.dot(Cj_invy))
        return float(likelihood)

    def invert_covariance(self, gradient=False, nonlinear =False, kernel=linear_kernel):
        q = self.latent_dim
        Nj = len(self.rated_items)
        Xj = np.asmatrix(self.X[self.rated_items, :])
        # Xj = np.asmatrix(self.X[int(self.rated_items), :])
        yj = np.asmatrix(self.y).T
        s_n = self.white_variance
        s_w = self.lin_variance
        s_b = self.bias_variance
        sigNoise = s_w / s_n

        if Nj > q and not nonlinear: # we use the matrix inversion lemma
            XTX = Xj.T * Xj
            B = np.eye(q) + sigNoise * XTX
            Binv = np.linalg.pinv(B)
            _, logdetB = np.linalg.slogdet(B)
            if gradient:
                AinvX = (Xj - sigNoise * Xj * (Binv * XTX)) / s_n
                AinvTr = (Nj - sigNoise * (np.multiply(Xj * Binv, Xj)).sum()) / s_n
            Ainvy = (yj - sigNoise * Xj * (Binv * (Xj.T * yj))) / s_n
            sumAinv = (np.ones((Nj, 1)) - sigNoise * Xj * (Binv * Xj.sum(axis=0).T)) / s_n  # this is Nx1
            sumAinvSum = sumAinv.sum()
            denom = 1 + s_b * sumAinvSum
            fact = s_b / denom
            if gradient:
                CinvX = AinvX - fact * sumAinv * (sumAinv.T * Xj)
                CinvSum = sumAinv - fact * sumAinv * sumAinvSum
                CinvTr = AinvTr - fact * sumAinv.T * sumAinv

            Cinvy = Ainvy - fact * sumAinv * float(sumAinv.T * yj)
            if not gradient:
                logdetA = Nj * np.log(s_n) + logdetB
                logdetC = logdetA + np.log(denom)

        else :
            C = s_w * kernel(Xj, Xj)
            C = C + s_b + s_n * np.eye(Nj)
            Cinv = np.linalg.pinv(C)
            Cinvy = Cinv * yj
            if gradient:
                CinvX = Cinv * Xj
                CinvTr = np.trace(Cinv)
                CinvSum = Cinv.sum(axis=1)
            else:
                _, logdetC = np.linalg.slogdet(C)

        if gradient:
            return Cinvy, CinvSum, CinvX, CinvTr
        else:
            return Cinvy, logdetC

    def log_likelihood_grad(self, ):
        """Computes the gradient of the log likelihood"""
        s_w = self.lin_variance
        s_b = self.bias_variance
        s_n = self.white_variance
        #print("invert_covariance----self.X[self.rated_items, :]:", self.X[[0, 1, 2], :])
        # print("self.rated_items:",self.rated_items,type(self.rated_items),self.rated_items.shape)
        #到这会出问题。
        #print("invert_covariance----self.X[self.rated_items, :]:", self.X[self.rated_items,:])

        yj = np.asmatrix(self.y).T
        Xj = np.asmatrix(self.X[self.rated_items, :])
        # Xj = np.asmatrix(self.X[int(self.rated_items), :])
        # print("invert_covariance----self.X[self.rated_items, :]:",self.X[self.rated_items, :])
        Cinvy, CinvSum, CinvX, CinvTr = self.invert_covariance(gradient=True)
        covGradX = 0.5 * (Cinvy * (Cinvy.T * Xj) - CinvX)
        gX = s_w * 2.0 * covGradX
        gsigma_w = np.multiply(covGradX, Xj).sum()
        CinvySum = Cinvy.sum()
        CinvSumSum = CinvSum.sum()
        gsigma_b = 0.5 * (CinvySum * CinvySum - CinvSumSum)
        gsigma_n = 0.5 * (Cinvy.T * Cinvy - CinvTr)
        return gX, float(gsigma_w), float(gsigma_b), float(gsigma_n)

    def objective(self):
        return -self.log_likelihood()


Fit代码

In [2]:
def fit(dataset, model, nb_iter=10, seed=42, momentum=0.9):
    data = dataset.get_df()
    param_init = np.zeros((1, 3))
    X_init = np.zeros(model.X.shape)
    for iter in range(nb_iter):
        print("iteration：", iter)
        tic = time.time()
        # 随机数要做什么？
        np.random.seed(seed=seed)
        state = np.random.get_state()
        #每次随机生成的都一样，详见test.py
        users = np.random.permutation(dataset.get_users())
        #print("users:",users.shape)

        for user in users:
            #print("begin user", user,  "=========================")
            toc = time.time()
            lr = 1e-4
            y = dataset.get_ratings_user(user)
            #print("the user:",user)
            #print("y:",y)
            # rated_items = dataset.get_items_user(user) - 1
            rated_items = dataset.get_items_user(user)
            #print("rated_items:",rated_items,"type:",type(rated_items))
            model.y = y
            #Ronchy将rated_items ndnumpy格式变成list
            rated_items = list(rated_items)
            #========================================
            model.rated_items = rated_items
            grad_X, grad_w, grad_b, grad_n = model.log_likelihood_grad()
            gradient_param = np.array([grad_w * model.lin_variance,
                               grad_b * model.bias_variance,
                               grad_n * model.white_variance])
            param = np.log(np.array([model.lin_variance,
                                     model.bias_variance,
                                     model.white_variance]))
            # update X
            X = X_init[rated_items, :]
            ar = lr * 10
            X = X * momentum + grad_X * ar
            X_init[rated_items, :] = X
            model.X[rated_items, :] = model.X[rated_items, :] + X

            # update variances
            param_init = param_init * momentum + gradient_param * lr
            param = param + param_init
            model.lin_variance = np.exp(param[0, 0])
            model.bias_variance = np.exp(param[0, 1])
            model.white_variance = np.exp(param[0, 2])
            #print("end user", user, "=========================")

        print("end iteration", iter,  "=========================")
        print("duration iteration", time.time() - tic)
    return model

Predict代码


In [3]:
def predict(user, test_items, model, dataset):
    y = dataset.get_ratings_user(user)
    # rated_items = dataset.get_items_user(user) - 1
    rated_items = dataset.get_items_user(user)
    rated_items = list(rated_items)
    test_items = list(test_items)
    model.rated_items = rated_items
    model.y = y
    X_test = np.asmatrix(model.X[test_items, :])
    X = np.asmatrix(model.X[model.rated_items, :])
    print("model.X[test_items, :]",model.X[test_items, :].shape)
    print("model.X[model.rated_items, :]",model.X[model.rated_items, :].shape)
    print("X",X.shape)
    print("X_test:",X_test.shape)
    Cinvy, CinvSum, CinvX, CinvTr = model.invert_covariance(gradient=True)
    mean = model.lin_variance* X_test*(X.T*Cinvy) + Cinvy.sum() * model.bias_variance
    return mean

执行代码


In [4]:
def perf_weak(dataset, base_dim=5):
    print('Fetch data set...')
    # if dataset.dataset == "movielens":
    #     norm_coeff = 1.6
    #     print("norm_coeff:1.6------------")
    # else :
    #     norm_coeff = 6.67
    norm_coeff = 1.6
    print('Data set fetched')
    # print("Dataset desctiption", dataset.get_description())
    model_init = GpMf(latent_dim=base_dim, nb_data=dataset.item_index_range)
    print('Fit the model...')
    model = fit(dataset=dataset, model=model_init)
    print('Model fitted')
    predictions = []
    true_ratings = []
    test_users = dataset.get_users_test()

    nb_users_test = len(test_users)
    print("nb_users_test", nb_users_test)
    count = 0
    for user in test_users:
        prediction = predict(user, dataset.get_item_test(user), model, dataset)
        if prediction > dataset.high_rating:
            prediction = dataset.high_rating
        if prediction < dataset.low_rating:
            prediction = dataset.low_rating
        predictions.append(prediction)
        rating = dataset.get_rating_test(user)
        true_ratings.append(rating)
        count += 1
        #print(count, "over ", nb_users_test, "users")
    predictions = np.asarray(predictions)
    true_ratings = np.asarray(true_ratings)
    #误差计算
    rmse = np.linalg.norm(predictions - true_ratings) / np.sqrt(nb_users_test)
    nmae = np.sum(np.abs(true_ratings - predictions)) * 1. / (len(predictions) * norm_coeff)
    print("rmse", rmse)
    print("nmae", nmae)
    return float(rmse), float(nmae)

画图


In [5]:
def plot_errors_vs_latent_dims():
    base_dims = range(3, 10)  #latent feature ,3的时候最好
    rmse_res = []
        #0.9192805345815771, 0.9191762806556765, 0.9298582192495648, 0.9264887573748214, 0.942721413670847,
        #0.9464276649204383, 0.9619938797137044, 0.959524838407498, 0.9637647705458857, 0.9654112679112156,
        #0.964014562524729, 0.9828162923399141, 0.9791815989138641, 0.9755912980143179, 0.9909916313775403]
    nmae_res = []
        #0.44971496453721255, 0.44916990674349777, 0.4520802986328138, 0.45121143493891314, 0.45841600020196077,
        #0.4600831341641973, 0.46819195277564857, 0.4653186575931832, 0.4675059307434209, 0.46521752440367997,
        #0.4693989965956004, 0.4749311584164483, 0.47302261002602103, 0.47072878663284334, 0.4775780782441432]

    if not len(rmse_res):
        dataset_Corner = DataSet(dataset="Corner", size="14")
        for dim in base_dims:
            print("the latent-feature:",dim)
            (rmse, nmae) = perf_weak(dataset=dataset_Corner, base_dim=dim)
            rmse_res.append(rmse)
            nmae_res.append(nmae)

        print(rmse_res)
        print(nmae_res)

    plt.figure()
    plt.plot(base_dims, rmse_res, marker='.')
    plt.grid()
    plt.xlabel('Number of latent dimensions ($r$)')
    plt.ylabel('RMSE (MovieLens 100k)')
    plt.show()

    plt.figure()
    plt.plot(base_dims, nmae_res, marker='*')
    plt.xlabel('Number of latent dimensions ($r$)')
    plt.ylabel('NMAE (MovieLens 100k)')
    plt.grid()
    plt.show()

if __name__ == "__main__":
    print('START')
    #perf_weak(dataset=DataSet(dataset="Corner", size="14"))

    # plot_errors_vs_latent_dims()
    
    
    print('END')


In [6]:
dataset = DataSet(dataset="Corner", size="14")
base_dim = 5

print('Fetch data set...')
# if dataset.dataset == "movielens":
#     norm_coeff = 1.6
#     print("norm_coeff:1.6------------")
# else :
#     norm_coeff = 6.67
norm_coeff = 1.6
print('Data set fetched')
# print("Dataset desctiption", dataset.get_description())
model_init = GpMf(latent_dim=base_dim, nb_data=dataset.item_index_range)
print('Fit the model...')
model = fit(dataset=dataset, model=model_init)
print('Model fitted')

self.low_rating: 2.301 self.high_rating: 8.292
self.item_index_range 13
self.df_train:        row col  value
0        0   0  3.335
1        1   0  3.311
2        2   0  4.600
3        3   0  5.083
4        4   0  6.327
...    ...  ..    ...
4207  1399   2  4.459
4208  1400   2  5.777
4209  1401   2  4.542
4210  1402   2  3.776
4211  1403   2  5.067

[4212 rows x 3 columns]
self.df_test:         row col  value
4212      0   3  2.694
4213      1   3  5.445
4214      2   3  2.642
4215      3   3  5.798
4216      4   3  3.563
...     ...  ..    ...
18247  1399  12  5.635
18248  1400  12  4.573
18249  1401  12  7.132
18250  1402  12  6.569
18251  1403  12  4.208

[14040 rows x 3 columns]
Fetch data set...
Data set fetched
self.X: [[ 0.01076326  0.00862603  0.00557698 -0.00791951 -0.00562826]
 [ 0.01735066  0.01020742  0.00540768  0.00721562  0.00021828]
 [-0.0110288  -0.01191749  0.01077815 -0.0040639  -0.00858522]
 [ 0.01874841 -0.01474276 -0.01172379  0.00983096  0.01405976]
 [ 0.00399321

做Predict

In [7]:
predictions = []
true_ratings = []
test_users = dataset.get_users_test()

nb_users_test = len(test_users)
print("nb_users_test", nb_users_test)
count = 0
for user in test_users:
    prediction = predict(user, dataset.get_item_test(user), model, dataset)
    
    #print("dataset.get_item_test(user):",dataset.get_item_test(user),dataset.get_item_test(user).shape)
    #print("prediction:",prediction,type(prediction))
#numpy 如何将大于或小于阈值的值替换为给定的阈值
#https://www.jianshu.com/p/a454b0657c4a
#     if prediction > dataset.high_rating:
#         prediction = dataset.high_rating
#     if prediction < dataset.low_rating:
#         prediction = dataset.low_rating
    predictions.append(prediction)
    rating = dataset.get_rating_test(user)
    true_ratings.append(rating)
    count += 1
    #print(count, "over ", nb_users_test, "users")
predictions = np.asarray(predictions)
predictions= predictions.reshape(predictions.shape[0],-1)

true_ratings = np.asarray(true_ratings)

print("predictions:",predictions,predictions.shape)
print("true_ratings:",true_ratings,true_ratings.shape)

#误差计算
rmse = np.linalg.norm(predictions - true_ratings) / np.sqrt(nb_users_test)
nmae = np.sum(np.abs(true_ratings - predictions)) * 1. / (len(predictions) * norm_coeff)
print("rmse", rmse)
print("nmae", nmae)

nb_users_test 1404
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

...

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 

model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X (3, 5)
X_test: (10, 5)
model.X[test_items, :] (10, 5)
model.X[model.rated_items, :] (3, 5)
X 